In [371]:
import sys
# sys.path.append('/mnt/6088FEB688FE89B4/Users/arsen/Desktop/code/git/python-utils-ak')
sys.path.append(r'C:\Users\arsen\Desktop\code\git\python-utils-ak')
from utils_ak.reflexion import load as _load

def _import_globals(module):
    for x in dir(module):
        if not x.startswith('_'):
            globals()[x] = getattr(module, x)
            
def load(module_obj, module_name=None, reload=False, import_globals=False):
    module = _load(module_obj, module_name, reload, import_globals=False)
    if import_globals:
        _import_globals(module)
    return module

In [501]:
load('src/time.py', import_globals=True, reload=True)
load('src/block.py', import_globals=True, reload=True)
load('src/color.py', import_globals=True, reload=True)
load('src/drawing.py', import_globals=True, reload=True)

<module 'src/drawing.py' from 'src/drawing.py'>

# Models

In [373]:
request = {'Моцарелла Фиор ди Латте в воде "Красная птица", 45%, 0,125 кг, ф/п': 1, 
          'Моцарелла Фиор Ди Латте в воде "Pretto", 45%, 0,125 кг, ф/п, (8 шт)': 1}

In [374]:
import pandas as pd

In [375]:
df = pd.DataFrame(request.items(), columns=['name', 'qty'])
df

,name,qty
0,"Моцарелла Фиор ди Латте в воде ""Красная птица""...",1
1,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, ...",1


In [296]:
import sys
# sys.path.append('/mnt/6088FEB688FE89B4/Users/arsen/Desktop/code/git/2020.10-umalat/umalat')
sys.path.append(r'C:\Users\arsen\Desktop\code\git\2020.10-umalat\umalat')


In [297]:
from app.models import *
from app.models import Boiling as BoilingModel

In [298]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# sqlite_filepath = '/home/akadaner/Desktop/data.sqlite'
sqlite_filepath = r"C:\Users\arsen\Desktop\code\git\2020.10-umalat\umalat\data.sqlite"
engine = create_engine(f"sqlite:///{sqlite_filepath}")
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [299]:
df['sku'] = df['name'].apply(lambda s: session.query(SKU).filter(SKU.name == s).first())

In [300]:
df

,name,qty,sku
0,"Моцарелла Фиор ди Латте в воде ""Красная птица""...",1,<SKU 27>
1,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, ...",1,<SKU 24>


In [301]:
sku = df.iloc[0]['sku']

In [302]:
session.query(BoilingModel).first()

<Boiling 1>

In [303]:
df['boiling'] = df['sku'].apply(lambda sku: session.query(BoilingModel).filter(BoilingModel.id == sku.boiling_id).first())

In [304]:
df

,name,qty,sku,boiling
0,"Моцарелла Фиор ди Латте в воде ""Красная птица""...",1,<SKU 27>,<Boiling 6>
1,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, ...",1,<SKU 24>,<Boiling 6>


In [305]:
lines = ['cheese_maker', 'melting', 'termizator', 'packing']

# Block

In [512]:
ROWS = {'water_1': 6, 'water_2': 9, 'salt_1': 15, 'salt_2': 18, 'water_melting': 24, 'water_packing': 29, 'salt_melting': 33, 'salt_packing': 46}
# ROWS = {'water_1': 1, 'water_2': 3, 'water_melting': 5, 'water_packing': 8}

def cast_row(obj):
    if isinstance(obj, int):
        return obj
    elif isinstance(obj, str):
        return ROWS[obj]
    else:
        raise Exception('Unknown object type')

In [513]:
style = {
    'pouring': {'beg_time': '01:00'},
    'termizator': {'text': '{block_num} налив'},
    'pouring_name': {'text': '{name}'},
    'pouring_and_fermenting': {'text': 'налив/внесение закваски', 'color': 'orange'},
    'soldification': {'text': 'схватка', 'color': 'yellow'},
    'cutting': {'text': 'резка/обсушка', 'color': '#92d050'},
    'pouring_off': {'text': 'слив', 'color': 'red'},
    'melting': {'beg_time': '07:00', 'y': cast_row('water_melting')},
    'melting_label': {'text': '{block_num}'},
    'melting_name': {'text': '{name}'},
    'serving': {'color': 'orange', 'text': 'подача и вымешивание'},
    'melting_process': {'color': 'orange', 'text': 'плавление {speed} кг/ч'},
    'cooling1': {'text': 'охлаж1 <{size} * 5> мин'},
    'cooling2': {'text': 'охлаж2 <{size} * 5> мин'},
    'packing': {'beg_time': '07:00', 'y': cast_row('water_packing')},
    'packing_label': {'text': '{block_num}'},
    'packing_name': {'text': '{name}'},
    'packing_brand': {'color': '#f2dcdb', 'text': '{name}'},
    'configuration': {'color': 'red'}
}

# Advanced Validator

In [521]:
def atom_validate(b1, b2):
#     print(b1['class'], b2['class'])
    b1, b2 = min([b1, b2], key=lambda b: str(b.interval)), max([b1, b2], key=lambda b: str(b.interval))
    
    if b1['class'] == b2['class'] == 'termizator':
        validate_disjoint(b1, b2)
    
    if b1['line'] == b2['line']:
        if {b1['class'], b2['class']} in [{'pouring'}, {'melting'}, {'packing'}]:
            validate_disjoint(b1, b2)
        
        if {b1['class'], b2['class']} in [{'packing', 'cooling1'}, {'packing', 'cooling2'}]:
            validate_disjoint(b1, b2)
    
def atom_validator(b1, b2):
    for atom_b1 in b1.flatten():
        for atom_b2 in b2.flatten():
            atom_validate(atom_b1, atom_b2)    

In [522]:
root = Block(t=10)
block1 = Block('pouring', size=5, line=1)
block1.make('termizator', size=1)
block2 = Block('pouring', size=5, line=2)
block2.make('termizator', size=1)
root.push(block1)
root.push(block2, validator=atom_validator)
root

block {'t': 10, 'class': 'block'}
  pouring {'size': 5, 'line': 1, 'class': 'pouring', 't': 0}
    termizator {'size': 1, 'class': 'termizator', 't': 0}
  pouring {'size': 5, 'line': 2, 'class': 'pouring', 't': 1}
    termizator {'size': 1, 'class': 'termizator', 't': 0}

In [523]:

def make_boiling(LINE):
    boiling = Block('boiling', block_num=12, line=LINE)
    
    total_size = 27
    with boiling.make('pouring', y=cast_row(f'water_{LINE}'), size=total_size) as pouring:
        with pouring.make(y=0, method='add') as b:
            b.make('termizator', size=6, block_num=12)
            b.make('pouring_name', size=total_size - 6, name='чильеджина  3,6 альче  8500кг')
        with pouring.make(y=1, method='add') as b:
            b.make('pouring_and_fermenting', size=8)
            b.make('soldification', size=7)
            b.make('cutting', size=7)
            b.make('pouring_off', size=3)
            b.make('extra', size=total_size - 8 - 8 - 7 - 3)

    boiling.make('drenator', size=cast_t('04:00'), visible=False)

    total_size = 25
    with boiling.make('melting_and_packing', y=10, size=total_size, line='1') as melting:
        with melting.make(y=0, method='add') as b:
            b.make('melting_label', size=3)
            b.make('melting_name', size=total_size - 3, name='чильеджина 5кг/сердечки/фдл 0,125(безлактозная)/0,1')
        with melting.make(y=1, method='add') as b:
            b.make('serving', size=6)
            b.make('melting_process', size=total_size - 6, speed=900)
        with melting.make(y=2, method='add') as b:
            b.make(size=6, visible=False)

            b.make('cooling1', size=5)
            b.make('cooling2', size=10)
            
            with b.make('packing', size=20) as packing:
                with packing.make(y=0, method='add') as b:
                    b.make('packing_label', size=3)
                    b.make('packing_name', size=total_size - 3, name='чильеджина 5кг/сердечки/фдл 0,125(безлактозная)/0,1')
                with packing.make(y=1, method='add') as b:
                    b.make('packing_brand', size=total_size, name='безлактозная/претто')
                with packing.make(y=2, method='add') as b:
                    b.make('configuration', size=3, visible=False)
                    b.make('configuration', size=1)
                    b.make('configuration', size=5, visible=False)
                    b.make('configuration', size=1)
    return boiling

In [524]:
root

block {'t': 10, 'class': 'block'}
  pouring {'size': 5, 'line': 1, 'class': 'pouring', 't': 0}
    termizator {'size': 1, 'class': 'termizator', 't': 0}
  pouring {'size': 5, 'line': 2, 'class': 'pouring', 't': 1}
    termizator {'size': 1, 'class': 'termizator', 't': 0}

In [525]:
root = Block('root', t=cast_t('01:00'), index_width=4)
boiling1 = make_boiling('1')
boiling2 = make_boiling('2')
root.push(boiling1)
root.push(boiling2, validator=atom_validator)
root

root {'t': 12, 'index_width': 4, 'class': 'root'}
  boiling {'block_num': 12, 'line': '1', 'class': 'boiling', 't': 0}
    pouring {'y': 6, 'size': 27, 'class': 'pouring', 't': 0}
      block {'y': 0, 'class': 'block'}
        termizator {'size': 6, 'block_num': 12, 'class': 'termizator', 't': 0}
        pouring_name {'size': 21, 'name': 'чильеджина  3,6 альче  8500кг', 'class': 'pouring_name', 't': 6}
      block {'y': 1, 'class': 'block'}
        pouring_and_fermenting {'size': 8, 'class': 'pouring_and_fermenting', 't': 0}
        soldification {'size': 7, 'class': 'soldification', 't': 8}
        cutting {'size': 7, 'class': 'cutting', 't': 15}
        pouring_off {'size': 3, 'class': 'pouring_off', 't': 22}
        extra {'size': 1, 'class': 'extra', 't': 25}
    drenator {'size': 48, 'visible': False, 'class': 'drenator', 't': 27}
    melting {'y': 10, 'size': 25, 'line': '1', 'class': 'melting', 't': 75}
      block {'y': 0, 'class': 'block'}
        melting_label {'size': 3, 'cl

In [526]:
# work_book, sheet = init_sheet()
work_book, sheet = init_template_sheet()
# update styles
for b in root:
    if b.block_class in style:
        b.local_props.update(style[b.block_class])
draw(sheet, root)
work_book.save('output.xlsx')

<Worksheet "планирование по цехам Copy"> 5 6 6 1 12 налив #ffffff
<Worksheet "планирование по цехам Copy"> 11 6 21 1 чильеджина  3,6 альче  8500кг #ffffff
<Worksheet "планирование по цехам Copy"> 5 7 8 1 налив/внесение закваски #ffa500
<Worksheet "планирование по цехам Copy"> 13 7 7 1 схватка #ffff00
<Worksheet "планирование по цехам Copy"> 20 7 7 1 резка/обсушка #92d050
<Worksheet "планирование по цехам Copy"> 27 7 3 1 слив #ff0000
<Worksheet "планирование по цехам Copy"> 30 7 1 1  #ffffff
<Worksheet "планирование по цехам Copy"> 8 24 3 1 12 #ffffff
<Worksheet "планирование по цехам Copy"> 11 24 22 1 чильеджина 5кг/сердечки/фдл 0,125(безлактозная)/0,1 #ffffff
<Worksheet "планирование по цехам Copy"> 8 25 6 1 подача и вымешивание #ffa500
<Worksheet "планирование по цехам Copy"> 14 25 19 1 плавление 900 кг/ч #ffa500
<Worksheet "планирование по цехам Copy"> 14 26 5 1 охлаж1 25 мин #ffffff
<Worksheet "планирование по цехам Copy"> 19 26 10 1 охлаж2 50 мин #ffffff
<Worksheet "планирование п

ValueError: Min value is 1

In [520]:
root

root {'t': 12, 'index_width': 4, 'class': 'root'}
  boiling {'block_num': 12, 'line': '1', 'class': 'boiling', 't': 0}
    pouring {'y': 6, 'size': 27, 'class': 'pouring', 't': 0, 'beg_time': '01:00'}
      block {'y': 0, 'class': 'block'}
        termizator {'size': 6, 'block_num': 12, 'class': 'termizator', 't': 0, 'text': '{block_num} налив'}
        pouring_name {'size': 21, 'name': 'чильеджина  3,6 альче  8500кг', 'class': 'pouring_name', 't': 6, 'text': '{name}'}
      block {'y': 1, 'class': 'block'}
        pouring_and_fermenting {'size': 8, 'class': 'pouring_and_fermenting', 't': 0, 'text': 'налив/внесение закваски', 'color': 'orange'}
        soldification {'size': 7, 'class': 'soldification', 't': 8, 'text': 'схватка', 'color': 'yellow'}
        cutting {'size': 7, 'class': 'cutting', 't': 15, 'text': 'резка/обсушка', 'color': '#92d050'}
        pouring_off {'size': 3, 'class': 'pouring_off', 't': 22, 'text': 'слив', 'color': 'red'}
        extra {'size': 1, 'class': 'extra'